In [1]:
# !pip install openpyxl
# !pip install python-docx

In [2]:
import pandas as pd
import os
import docx
import sys
import traceback
import datetime
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import time
import logging

## Coded string reader function

In [3]:
def read(doc_obj):
    '''
    This function reads the coded string in a docx file to a dataframe
    '''
    
    try:
        # with open(read_path, 'r') as file:
        #     coded_string = file.read()
        if len(doc_obj.paragraphs) > 1:
            print(f"At this action {doc_obj.paragraphs[1].text.split(',')[0]}, there is an issue. Please Check!")
            sys.exit(1)
        else:                  
            for paragraph in doc_obj.paragraphs:
                coded_string = paragraph.text
                coded_string_list = coded_string.split(',')
                df = pd.DataFrame(coded_string_list, columns = ['strings'])
                df[['team', 'jersey_number', 'action', 'notation', 'start_grid', 'end_grid', 'timestamp', 'foot', 'special_attribute', 'half']] = df['strings'].str.split('-', expand = True)
                df = df.drop(columns = ['strings'])
                return df
                
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        traceback.print_exc()


In [4]:
# read_path = r'..\read_string\rstring.txt'
doc_obj = docx.Document(r"..\read_string\rstring.docx")
df = read(doc_obj)

## Renumeration log function

In [5]:
match_idd = int(input('Match ID please... = '))

def renumeration_func():
    '''
    This function creates a renumeration excel file as per the inputs given.
    It checks the match id to make sure no matches are duplicated.
    '''
    try:

        analyst_names = {
            1 : 'Sreyas',
            2 : 'Boni',
            3 : 'Arpit',
            4 : 'Sudhanva'
        }
        
        write_path_xlsx = r'..\write_string\renumeration.xlsx'
        initial_input = input('Do you wish to add match details? (y/n)')

        # Taking the match details from the user
        if initial_input.lower() == 'y':
            analyst_id = int(input(f'Choose the Analyst ID\n{analyst_names} = '))
            analyst_name = analyst_names[analyst_id]
            team_a_name = input('Team A name = ')
            team_b_name = input('Team B name = ')
            match_id = match_idd
            game_time = int(input('Game time?\nType 1 for 90 minutes\nType 2 for 45 minutes\nType 3 for less than 60 minutes\n'))
            current_date = datetime.date.today()
        
            if game_time == 1:
                renumeration = 500
                game_time = 90
            elif game_time == 2:
                renumeration = 250
                game_time = 45
            elif game_time == 3:
                renumeration = 300
                game_time = 'Less than 60'
            else:
                print('You have chosen an invalid option. Try again!')
        
            data = {
                    'team_a_name' : team_a_name,
                    'team_b_name' : team_b_name,
                    'match_id' : match_id,
                    'game_time' : game_time,
                    'current_date' : current_date,
                    'renumeration' : renumeration
                }        
            
            # There are 4 possible scenarios here
            # 1. The file exists
            if os.path.exists(write_path_xlsx):
                wb = load_workbook(write_path_xlsx)
                sheet_name = analyst_name
                sheet_exists = sheet_name in wb.sheetnames
                
                # 2. File exists but sheet doesn't  
                if not sheet_exists:                
                    ws = wb.create_sheet(sheet_name)                
                    renumeration_df = pd.DataFrame(data, index = [0])
                    for r in dataframe_to_rows(renumeration_df, index=False, header=True):
                        ws.append(r)
                    for cell in ws[1]:
                        cell.style = 'Pandas'
                    wb.save(write_path_xlsx)
                    
                # 3. File and sheet exists
                else:                  
                    renumeration_df = pd.read_excel(write_path_xlsx, sheet_name = analyst_name)
                    renumeration_df.loc[len(renumeration_df)] = data
                    if renumeration_df['match_id'].duplicated().sum() > 0:
                        display(renumeration_df[renumeration_df['match_id'].duplicated()])
                        print('This match id seems to be a duplicate. Please check!')
                        # renumeration_df = renumeration_df.drop_duplicates(subset = ['match_id'])
                    else:
                        renumeration_df = pd.DataFrame(data, index = [0])
                        for r in dataframe_to_rows(renumeration_df, index=False, header=False):
                            ws = wb[sheet_name]
                            ws.append(r)
                        wb.save(write_path_xlsx)
                        
            # 4. File does not exists 
            else:
                renumeration_df = pd.DataFrame(data, index = [0])
                renumeration_df.to_excel(write_path_xlsx, sheet_name = analyst_name, index = False)     
                
           
            
                
        else:
            print('Proceeding without filling details')

    except PermissionError as e:
        print(f'{e}\nThe renumeration xlsx file might be open. Please close it and try again')

    except Exception as e:
        print(f'An error occured : {e}')

Match ID please... =  1108


In [6]:
renumeration_func()

Do you wish to add match details? (y/n) y
Choose the Analyst ID
{1: 'Sreyas', 2: 'Boni', 3: 'Arpit', 4: 'Sudhanva'} =  1
Team A name =  Rajasthan United
Team B name =  Churchill Brothers
Game time?
Type 1 for 90 minutes
Type 2 for 45 minutes
Type 3 for less than 60 minutes
 1


## Logger function

In [7]:
def logger_setup(filename):
    logging.basicConfig(filename=filename, filemode="w", level=logging.DEBUG)
    log_obj = logging.getLogger()
    return log_obj

In [8]:
logs_path = r'..\logs'
if not os.path.exists(logs_path):
    os.makedirs(logs_path)

logger_obj = logger_setup("../logs/QC_Match_ID_" + str(match_idd) + "_Time_" + time.strftime("%H-%M-%S") + ".log")

## Dataframe to coded string function

In [9]:
def write(write_path):
    '''
    This function creates a coded string from a dataframe
    '''
    
    if not 'Combined' in df.columns:
        df['Combined'] = df.apply(lambda row: '-'.join(row), axis=1)
        cs_list = list()
        df['Combined'].apply(lambda x : cs_list.append(x))
        
        if not os.path.exists(write_path):
            with open(write_path, 'a') as file:
                for item in cs_list:
                    if item == cs_list[-1]:
                        file.write(item)
                    else:
                        file.write(item + ',')

            print("wstring file successfully created.")
    
        else:
            print("This file already exists")

    else:
        print("Read the file again. Combined column already exists.")

In [10]:
write_path = '..\write_string\wstring.txt'
# write(write_path)

## Dataframe to coded string function mini

In [11]:
def write_log(log_df):
    '''
    This function creates a coded string from a dataframe for logger object
    '''
    log_df = log_df.copy()
    if not 'Combined' in log_df.columns:
        log_df['Combined'] = log_df.apply(lambda row: '-'.join(row), axis=1)
        cs_list = list()
        log_df['Combined'].apply(lambda x : cs_list.append(x))
        return cs_list

## Appropriate fouls QC function

In [12]:
def appropriate_foul():
    '''
    This function checks if 
    1. A non-defensive action(other than ST, SL, AD, GD) was given a foul
    2. A successful defensive action was tagged as a foul(need to check for yc and rc)
    '''
    non_df_action = df[~(df['action'].isin(['ST', 'SL', 'AD', 'GD'])) & (df['special_attribute'].isin(['F', 'YC', 'RC']))]
    non_df_action_foul = non_df_action['action'].count()

    successful_df_action = df[(df['action'].isin(['ST', 'SL', 'AD', 'GD']))  & (df['notation'] == '1') & (df['special_attribute'].isin(['F', 'YC', 'RC']))]
    successful_df_action_foul = successful_df_action['action'].count()
    
    if (non_df_action_foul != 0) | (successful_df_action_foul != 0):
        if (non_df_action_foul != 0):
            print('A non defensive action was tagged as a foul. Please check!')
            display(non_df_action)
            logger_obj.info(f"\n\nA non defensive action was tagged as a foul. Please check! \n{write_log(non_df_action)}\n\n" + '*'*100)
        
        if (successful_df_action_foul != 0):
            print('A successful defensive action was tagged as a foul. Please check!')
            display(successful_df_action)
            logger_obj.info(f"\n\nA successful defensive action was tagged as a foul. Please check! \n{write_log(successful_df_action)}\n\n" + '*'*100)
    else:
        print('Appropriate foul QC done.')


In [13]:
appropriate_foul()

A non defensive action was tagged as a foul. Please check!


,team,jersey_number,action,notation,start_grid,end_grid,timestamp,foot,special_attribute,half
1377,B,33,SP,1,54,63,1:47:29,R,YC,SHN


## Freekick-Penalty = Fouls QC function

In [14]:
def fk_pk_foul_check():
    '''
    This function finds the misbehaviour foul counts, subtracts them from total fouls and checks if total fouls equal
    total freekick-penalty
    '''
    # finding misbehaviour foul count
    misbehaviour_foul_a = 0
    misbehaviour_foul_b = 0
    
    st_0_list = df[(df['action'] == 'ST') & (df['notation'] == '0') & (df['special_attribute'].isin(['YC', 'RC']))].index
    for index in st_0_list:
        if ((index == df.index[-1]) or (df.loc[index + 1, 'action'] != 'XST')) & (df.loc[index, 'team'] == 'A'):
            misbehaviour_foul_a += 1
        elif ((index == df.index[-1]) or (df.loc[index + 1, 'action'] != 'XST')) & (df.loc[index, 'team'] == 'B'):
            misbehaviour_foul_b += 1

    # finding fk-pk and fouls count
    teamb_foul = df[(df['team'] == 'B') & ((df['action'].isin(['HB', 'OFF'])) | (df['special_attribute'].isin(['F', 'YC', 'RC'])))]['action'].count()
    teamb_foul = teamb_foul - misbehaviour_foul_b
    teamb_fk_pk = df[(df['team'] == 'B') & (df['special_attribute'].isin(['FK', 'PK']))]['action'].count()
    
    teama_foul = df[(df['team'] == 'A') & ((df['action'].isin(['HB', 'OFF'])) | (df['special_attribute'].isin(['F', 'YC', 'RC'])))]['action'].count()
    teama_foul = teama_foul - misbehaviour_foul_a
    teama_fk_pk = df[(df['team'] == 'A') & (df['special_attribute'].isin(['FK', 'PK']))]['action'].count()

    # verifying if the foul and fk-pk count match
    if (teamb_foul == teama_fk_pk) & (teama_foul == teamb_fk_pk):
        print("Foul to fk-pk QC done")
        logger_obj.info('\n\nFoul to fk-pk QC done\n\n' + '*'*100)
    else:
        print("Foul to fk-pk not equal")
        print(f"Team A FK-PK = {teama_fk_pk}, Team B Fouls = {teamb_foul}")
        print(f"Team B FK-PK = {teamb_fk_pk}, Team A Fouls = {teama_foul}")
        logger_obj.info(f'''\n\n
        Foul to fk-pk not equal
        Team A FK-PK = {teama_fk_pk}, Team B Fouls = {teamb_foul}
        Team B FK-PK = {teamb_fk_pk}, Team A Fouls = {teama_foul}\n
        ''')
    
        teamB = (teamb_foul == teama_fk_pk)
        teamA = (teama_foul == teamb_fk_pk)
        if not (teamB | teamA):
            print('Both inequal. Check TMS')
        elif teamB:
            foul_team = 'A'
            fk_team = 'B'
        else:
            foul_team = 'B'
            fk_team = 'A'
    
        try:
            foul_mask = ( (df['team'] == foul_team) & ((df['action'].isin(['HB', 'OFF'])) | (df['special_attribute'].isin(['F', 'YC', 'RC']))) )
            fk_mask = ( (df['team'] == fk_team) & (df['special_attribute'].isin(['FK', 'PK'])) )
            display(df[foul_mask | fk_mask])
            
            logger_string_list = write_log(df[foul_mask | fk_mask])
            logger_string = ''
            for string in logger_string_list:
                logger_string = logger_string + string + '\n'
            logger_obj.info(f'\n\n{logger_string}\n\n' + '*'*100)
    
        except:
            pass

In [15]:
fk_pk_foul_check()

Foul to fk-pk not equal
Team A FK-PK = 21, Team B Fouls = 22
Team B FK-PK = 11, Team A Fouls = 11


,team,jersey_number,action,notation,start_grid,end_grid,timestamp,foot,special_attribute,half
8,B,9,OFF,1,23,X,0:22:03,R,X,FHN
9,A,31,LP,0,58,53,0:22:36,R,FK,FHN
47,B,9,AD,0,25,X,0:24:29,R,F,FHN
51,A,4,LP,1,56,13,0:24:51,R,FK,FHN
243,B,10,ST,0,23,X,0:33:50,R,F,FHN
245,A,31,LP,0,58,43,0:34:16,R,FK,FHN
309,B,8,ST,0,25,X,0:36:34,R,F,FHN
311,A,4,SP,1,56,56,0:36:43,R,FK,FHN
418,B,57,OFF,1,31,X,0:40:33,R,X,FHN
419,A,31,LP,0,40,6,0:41:01,R,FK,FHN


## Corner QC function

In [16]:
def corner_qc():
    """
    The purpose of this function
    1. If there are any corners which didn't start from the corner grids
    2. If there are any unassigned corners within the crosses
    
    """

    # To check if there were any passes which didn't start from the correct corner start grid(1, 10, 71, 80) designated as corners
    false_cn = df[(df['special_attribute'] == 'CN') & ~(df['start_grid'].isin(['01', '1', '10', '71', '80']))]
    false_cn_count = false_cn['action'].count()
    if false_cn_count != 0:
        print("This Corner started from a false start grid. Check!")
        display(false_cn)

        logger_obj.info('\n\nThis Corner started from a false start grid. Check!\n' + f'{write_log(false_cn)}\n\n' + '*'*100)
    else:
        print("No corner false start grid error")
        logger_obj.info("\n\nNo corner false start grid error\n\n" + '*'*100)

    

    print(f"\n{'*' * 90}\n")
    
    
    # To check if any crosses starting from (1, 10, 71, 80) are corners
    corner_in_cross = df[(df['special_attribute'] != 'CN') & (df['start_grid'].isin(['01', '1', '10', '71', '80'])) & (df['action'] == 'C')]
    corner_in_cross_count = corner_in_cross['action'].count()
    if corner_in_cross_count != 0:
        print(f"{corner_in_cross_count} cross found with corner grid. Check if they are crosses")
        display(corner_in_cross)

        logger_obj.info(f'\n\n{corner_in_cross_count} cross found with corner grid. Check if they are crosses\n'\
                        + f'{write_log(corner_in_cross)}\n\n' + '*'*100)
    else:
        print("No corners in crosses")
        logger_obj.info("\n\nNo corners in crosses\n\n" + '*'*100)

In [17]:
corner_qc()

No corner false start grid error

******************************************************************************************

No corners in crosses


## Key passes and Assist QC

In [18]:
index_list = df[df['action'].isin(['CS', 'LS'])].index

In [19]:
# for index in index_list:
#     display(df.loc[index-5:index])

## Goal Kick QC

In [20]:
def gk_qc():
    '''
    This function checks if the GK, GH and GT were taken from inside their respective grids or not
    '''

    # Checking if GK is taken from outside goal area
    wrong_goalkick = df[(df['special_attribute'] == 'GK') & ~(df['start_grid'].isin(['31', '41', '40', '50']))]
    wrong_goalkick_count = wrong_goalkick['action'].count()
    if wrong_goalkick_count > 0:
        print('GK QC-1\nThis goal kick is taken outside the goal area. Please check!')
        display(wrong_goalkick)   

        logger_obj.info('\n\nGK QC-1\nThis goal kick is taken outside the goal area. Please check!\n'\
                       + f'{write_log(wrong_goalkick)}\n' + '*'*100)
    else:
        print('GK QC-1 done.')
        logger_obj.info('\n\nGK QC-1 done.\n\n' + '*'*100)

    # Chceking GH or GT is taken from outside penalty area
    gk_d_grids = ['21','22','29','30','31','32','39','40','41','42','49','50','51','52','59','60']
    wrong_goalkeeper = df[(df['action'].isin(['GH', 'GT'])) & ~(df['start_grid'].isin(gk_d_grids))]
    wrong_goalkeeper_count = wrong_goalkeeper['action'].count()
    if wrong_goalkeeper_count > 0:
        print('GK QC-2\nThis GH or GT taken outside penalty area. Please check!')
        display(wrong_goalkeeper)

        logger_obj.info('\n\nGK QC-2\nThis GH or GT taken outside penalty area. Please check!\n'\
                       + f'{write_log(wrong_goalkeeper)}\n' + '*'*100)
        
    else:
        print('GK QC-2 done.')
        logger_obj.info('\n\nGK QC-2 done.\n\n' + '*'*100)

In [21]:
gk_qc()

GK QC-1 done.
GK QC-2 done.
